##### 본사시상 추출- 최종 결과파일로 후처리 수작업 필요

In [173]:
from openpyxl import load_workbook
import pandas as pd
import os
import GladUtil as glad
import re
p = re.compile('(\d|생명|라이프|본사|환수|월|영업|활성화|시상|\.|-|_|\(|\))')
def conv_life(lname):
    lname = re.sub(p,'',lname)
    return {'처브':'처브라이프',
             '미래':'미래에셋',
             '메트':'메트라이프'}.get(lname,lname+'생명').replace(' ','')

''' 본사시상금을 추출하는 스크립트
    입출력 파일명과 업적월,지급월을 수정한 후에 실행하면 출력파일에 시트가 추가됨
'''
업적월 = 202408
지급월 = 202409
if 업적월 > 202212:  #정산비율
    SHARE_RT = 20
    SHARE_RT_HQ = 0.6
else: 
    SHARE_RT = 30
    SHARE_RT_HQ = 0.9

infile = f'E:/글로벌백업(180329)/04.수수료작업/{업적월}시상/{업적월} KJ시상정산-배포.xlsx'
outfile = 'E:/글로벌백업(180329)/04.수수료작업/시상이자/본사시상정산_이자.xlsx'

sheet_nm = ['생보스팟','한화','KB','농협','MG','롯데','DB','현대','메리츠','흥국','AIG','삼성','처브','하나']

FIX_VAL = [업적월,지급월,'']

wb = load_workbook(infile,data_only=True)
ws = wb['한화']
df = pd.DataFrame(columns=['업적월','지급월','회사','지점','사원','시상금','비고'])

''' 각 시트에서 본사 시상금만 추출 ''' 
for sh in sheet_nm:
    ws = wb[sh]
    FIX_VAL[2] = sh
    
    st_yn = 'n'
    for cell in ws['a']:
        if cell.value:
            if '본사' in cell.value:  # 본사 시상금은 시트의 맨 아래에 있어야 함
                st_yn = 'y'
            if  st_yn =='y':
                if '본사' in cell.value or '글래드' in cell.value:  #본사시상명 또는 지점있는 row
                    r = cell.row
                    vals = []
                    if '본사' in cell.value:
                       titl = cell.value
                       vals = [ cell.value, ' ',' ',' ']
                    if '글래드' in cell.value:
                        vals = [ x.value for x in ws[r] if x.value !=None ]
                    val = [ vals[0],vals[1],vals[len(vals)-1] ]  #지점,이름,시상금
                    val.append(titl)
                    df.loc[len(df)] = FIX_VAL + val

##생보스팟에서추출한 경우 회사명을 변환함
df['회사']= [ conv_life(y) if x=='생보스팟' else x for (x,y) in zip(df['회사'],df['비고'])]
##인센티브 시상은 제외(이자계산안함)
df = df[~df['비고'].str.contains("인센|티브")]
##글래드라온은 공동정산에서 제외:22.8월부터
df = df[df['지점'] != '글래드라온']

#--공동정산 대상건만 추출
df2 = glad.Share_Proc(df,"공동","본사x")  # 본사 헤더는 추출하지 않음
df2['시상금'] = df2['시상금'].astype('int32',errors='ignore')  
df2 = df2[df2['시상금']!=0]
df2['내역'] = df2['비고'].apply(lambda x: '환수_'+''.join(re.findall('\d|\.',x)) 
              if '환수' in x else '지급_'+str(업적월))
df2.drop(columns='비고',inplace=True)
df2.sort_values(by=['내역','회사'],inplace=True)

w_mode = 'a' if os.path.isfile(outfile) else 'w'  #파일있으면 append mode
with pd.ExcelWriter(outfile, mode = w_mode, engine="openpyxl") as writer:
     df2.to_excel(writer, sheet_name=str(업적월), index=False)
        
print("완료.. 환수건을 해당월로 이동하는 아래 스크립트를 실행~~~")

C:\Users\admin\anaconda3\lib\site-packages\openpyxl\reader\excel.py:228: UserWarning: Conditional Formatting extension is not supported and will be removed
  ws_parser.bind_all()


완료.. 환수건을 해당월로 이동하는 아래 스크립트를 실행~~~


#####  환수건을 해당 업적월로 이동

In [174]:
# outfile ='E:/글로벌백업(180329)/04.수수료작업/시상이자/본사시상정산_이자 - 복사본 - 복사본 (2).xlsx'
copyfile= "E:/글로벌백업(180329)/04.수수료작업/시상이자/본사시상정산_이자_temp.xlsx"
import shutil

def make_ym(s, ym):
    if '환수' in s:
        yy = re.sub(r'[^0-9]', '', s.split('.')[0])  #년도와 월 .으로 구분되어 있어야
        mm = s.split('.')[1].rjust(2,'0')
        return '20'+yy+mm if len(yy) < 4 else yy+mm
    else:
        return str(ym)
    
shutil.copyfile(outfile, copyfile)
df = pd.read_excel(copyfile, sheet_name = None,header = 0)  
p = re.compile('\d{6}') 

#당월분
dft = df[str(업적월)]
#당월분에 대해 업적월 표시
dft['ym'] = dft['내역'].apply(lambda x : make_ym(x,업적월))
#당월분 중에서 환수건 추출
dft1 = dft[dft['ym'] != str(업적월)] 

# 환수건을 업적월로 이동
tot_cnt = 0
lst = dft['ym'].unique()
for i in lst:
    dd = dft1[dft1['ym'] == i]
    # print(i, '= ', len(df[i]) ,' +  ', len(dd), '건' )
    df[i] = pd.concat([df[i], dd.iloc[:,:7]])
    tot_cnt += len(dd)

#당월분 중에서 환수건 제거
df[str(업적월)] = dft[dft['ym'] == str(업적월)].iloc[:,:7]
print('시트이동 ', tot_cnt,'건  당월', len(dft),'->', len(df[str(업적월)]))

#파일을 append mode로 읽어서 해당 시트만 replace함
w_mode = 'a'
with pd.ExcelWriter(outfile, mode = w_mode, if_sheet_exists='replace', engine="openpyxl") as writer:
    [ df[i].to_excel(writer, sheet_name= str(i), index=False) for i in lst]

print('회사목록 확인=> ',list(dft['회사'].unique()))
print('end of job')

시트이동  106 건  당월 302 -> 196
회사목록 확인=>  ['DB', 'KB', 'MG', '롯데', '메리츠', '삼성', '삼성생명', '처브', '하나', '한화', '현대', '흥국']
end of job


##### 본사시상 2차년도 정산금-mp정산 생성

In [170]:
from openpyxl import load_workbook
import pandas as pd
import os
import GladUtil as glad
# 미정산: 2022.10~2023.2
# 업적월 = 202408
# SHARE_RT = 20
# SHARE_RT_HQ = 0.6
infile =f'E:/글로벌백업(180329)/04.수수료작업/시상이자/본사시상정산_2차년도_{업적월}.xlsx'
outfile = f'E:/글로벌백업(180329)/04.수수료작업/시상이자/본사시상정산_2차년도_{업적월}_mp.xlsx'

df_dict = pd.read_excel(infile,sheet_name=None)
df_1 = pd.DataFrame()

for sh,df in df_dict.items():
    if sh !='요약' and sh != '비공개':
        df2 = glad.Share_Proc(df,"공동","본사x") 
        df_1 = pd.concat([df_1, df2])

if df_1['보험사'].isna().sum() > 0:
    print('error =======> 보험사....')
##글래드라온은 공동정산에서 제외:22.8월부터
df_1 = df_1[(df_1['지점'] != '글래드라온') & (df_1['정산금'] !=0)]

# 정산비율 조정..23.11.10일 작업부터
# df_all = df_all[df_all['정산금'] !=0].sample(frac=0.9, random_state=1004)
df_all = pd.DataFrame()
for i in  df_1['보험사'].unique():
    df_temp = df_1[df_1['보험사'] ==i].sample(frac=0.88, random_state=10)
    df_all  = pd.concat([df_all, df_temp])

df_all = df_all.fillna(0)
df_all['작업일'] = df_all['작업일'].astype('str')

df_sum = pd.pivot_table(df_all,index=['지점'],columns=['업적월','보험사'],values='정산금',aggfunc='sum',fill_value=0, 
               margins=False, margins_name='Total').reset_index()

직영 = ['글래드레전드','글래드여주','글래드의정부','글래드제이엔제이','글래드씨티엠','글래드프라임-J&J']
GLAD_HQ = ['글래드','글래드HL']

df_sum['합계'] = df_sum.sum(axis=1, numeric_only=True)
df_sum['MP율'] = df_sum['지점'].apply( lambda x: SHARE_RT_HQ if x in GLAD_HQ else ( 0 if x in 직영 else SHARE_RT ))
df_sum['MP']   = round( df_sum['합계'] * df_sum['MP율'] / 100,0)
df_sum.loc[len(df_sum)] = df_sum.sum( numeric_only=True)
df_sum['지점'] = df_sum['지점'].fillna('합계')
df_sum.drop(columns=('MP율',''), inplace=True)

with pd.ExcelWriter(outfile,mode='w',date_format="YYYY-MM-DD") as writer:
    df_sum.to_excel(writer, sheet_name='요약')
    df_all.to_excel(writer, sheet_name="명세", index =False)
    workbook = writer.book
    format1 = workbook.add_format({'num_format': '#,##0'})
    worksheet = writer.sheets["요약"]
    worksheet.set_column('C:CC', None, format1)
    worksheet.set_column(0,0,0)
    worksheet.set_column('B:B',18)
    worksheet1 = writer.sheets["명세"]
    worksheet1.set_column('E:H', None, format1)
    worksheet.set_row(2, 2, None, {'hidden': True}) #multi-index에서 생성된 빈 row hidden처리
  
print('완료 :',df_all['정산금'].sum()/ df_1['정산금'].sum())

완료 : 0.8578153492341201


#### 공제내역 - 월별 공제금액 추출

In [ ]:
path = f'E:/글로벌백업(180329)/04.수수료작업/시상이자/백업/'
infile= f'{path}# 20230427_F_01_202111~202302_글로벌금융판매 본사시상금(영업활성화) 이자 공제_배포.xlsx'

df_dict = pd.read_excel(infile,sheet_name=None)
df_all = pd.DataFrame()

for sh,df in df_dict.items():
    # print(type(sh),sh)
    if '총괄별명세서' in sh:
        df2 = df
        ll = df2.iloc[1:2,:].fillna(0).values.tolist()[0]
        # print(sh, ll)
        idx = [i for i in range(len(ll)) if '공제 계' in str(ll[i])]
        if idx:
            idx_num = idx[0]
            ll_val = df2[df2['Unnamed: 1']=='GLAD'].fillna(0).values.tolist()[0][idx_num]
            print(sh,'=', ll_val, '컬럼=',idx_num)


 ### 성공,송내건만 추출--미사용

In [ ]:
from openpyxl import load_workbook
import pandas as pd
import os
import GladUtil as glad
업적월 = '2022010'

import shutil
infile =f'E:/글로벌백업(180329)/04.수수료작업/시상이자/본사시상정산_이자.xlsx'
outfile = f'E:/글로벌백업(180329)/04.수수료작업/시상이자/본사시상정산_이자_성공송내.xlsx'
shutil.copyfile(infile, outfile)

GLAD_HQ = ['글래드성공','글래드송내']
df_dict = pd.read_excel(outfile,sheet_name=None)

for sh,df in df_dict.items():
    if sh >= '202107' and sh <= 업적월:
        print(sh)
        df2 = df[df['지점'].isin(GLAD_HQ)]
        with pd.ExcelWriter(outfile, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
             df2.to_excel(writer, sh, index=False)

print('완료')

## 본사시상 추세

In [171]:
from openpyxl import load_workbook
import pandas as pd
import os
import GladUtil as glad

def read_data(ym):
    infile =f'E:/글로벌백업(180329)/04.수수료작업/시상이자/본사시상정산_2차년도_{ym}_mp.xlsx'

    df  = pd.read_excel(infile,sheet_name='요약')
    lst=list( df.columns)
    new_lst=[]
    for i,x in enumerate(lst):
        if "Unname" in str(x):
            new_lst.append(new_lst[i-1])
        else:
            new_lst.append(x)
        
    df.columns = new_lst
    cols = list(df.T.iloc[1])
    cols[:2] = ['업적월','보험사']
    df_new = df.T.iloc[2:]
    df_new.columns= cols
    df_new = df_new.reset_index().drop(columns='보험사').rename(columns=({'index':'업적월','업적월':'보험사'}))
    df_new= df_new.set_index(['업적월','보험사']).stack().reset_index()
    df_new.columns = ['업적월','보험사','지점','금액']
    df_new['보험사'].fillna('합계', inplace=True)
    df_new.insert(0,'지급월',ym)
    return df_new
    
years = ['2023','2024']
ym_list = [ i+str(j+1).rjust(2,'0') for i in years for j in range(12)]
#작업대상 기간 
ym_list = [ i for i in ym_list if i >= '202307' and i <'202408']
df_all = pd.DataFrame()
for ym in ym_list:
    # print(ym)
    df_all = pd.concat([df_all, read_data(ym)]) 

# df_all[(df_all['업적월']=='MP') & (df_all['보험사']=='합계')& (df_all['지점']=='합계')]
df_res = df_all[(df_all['보험사']!='합계') & (df_all['지점']!='합계')]
df_res['MP'] = [x*0.006 if y=='글래드' or y=='글래드HL' else x*0.2 for x,y in zip(df_res['금액'],df_res['지점']) ]
df_t = df_res.groupby(['지급월','업적월']).agg({'금액':'sum','MP':'sum'}).reset_index()
df_res = pd.concat([df_res,df_t])
df_res['지점'] = df_res['지점'].fillna('합계')
print('end')

end


C:\Users\admin\AppData\Local\Temp\ipykernel_1556\745353414.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['MP'] = [x*0.006 if y=='글래드' or y=='글래드HL' else x*0.2 for x,y in zip(df_res['금액'],df_res['지점']) ]


In [172]:
df_res.to_clipboard()

In [126]:
df_res = df_all[(df_all['보험사']!='합계') & (df_all['지점']!='합계')]
df_res['MP'] = [x*0.006 if y=='글래드' else x*0.2 for x,y in zip(df_res['금액'],df_res['지점']) ]


C:\Users\admin\AppData\Local\Temp\ipykernel_1556\520211474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['MP'] = [x*0.006 if y=='글래드' else x*0.2 for x,y in zip(df_res['금액'],df_res['지점']) ]


In [166]:
df_res[(df_res['지급월']=='202407') & (df_res['지점']=='합계')]['MP'].sum()

12043654.222407406

In [160]:
df_t

,지급월,업적월,금액,MP
0,202307,202108,37148.0,7.004488e+03
1,202307,202109,58030.25,1.063605e+04
2,202307,202110,47125.2,8.516286e+03
3,202307,202111,44770.65,8.372130e+03
4,202307,202112,16574.8,1.995760e+03
...,...,...,...,...
121,202407,202304,5864470,1.145912e+06
122,202407,202305,15270132.3,2.990997e+06
123,202407,202306,22723043.807407,4.214717e+06
124,202407,202304(분),6537240,1.239882e+06


In [147]:
df_res.groupby(['지급월','업적월']).agg({'금액':'sum','MP':'sum'}).reset_index()

,지급월,업적월,금액,MP
0,202307,202108,37148.0,7.004488e+03
1,202307,202109,58030.25,1.063605e+04
2,202307,202110,47125.2,8.516286e+03
3,202307,202111,44770.65,8.372130e+03
4,202307,202112,16574.8,1.995760e+03
...,...,...,...,...
121,202407,202304,5864470,1.145912e+06
122,202407,202305,15270132.3,2.990997e+06
123,202407,202306,22723043.807407,4.214717e+06
124,202407,202304(분),6537240,1.239882e+06
